<a href="https://colab.research.google.com/github/yuxuzi/Dash/blob/master/Edge_Stream_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Edge Stream Code Exam

##  Interpolate prices

Solution 1.  Since Scipy has interpolate.interp1d fuction. We can use it to do linear interpolation. It also has the extrapolate option.

Solution 2. Without usign scipy 

* Scan the list and ignore prices that less or equal to 0
* Find the neigbor index of the n
* Interpolate or extrapolate

Performance
* The time complexity is O(N) for scanning the lists of length N. 
* The space complexity is O(N)
* Early return makes code efficient
* The solution 2 matches the solution 1, and ten times faster for short list.
* The solution 1 may be efficient for large data.



In [ ]:
from typing import List
import numpy as np
from scipy import interpolate

def interpolate_prices0(n:int, instances:List[int], prices:List[float])->float:
    """
    interpolate using scipy.interpolate.
    :params:
      n : instance to find price
      instances: previous instances
      prices: previous prices
    :return:
      price for the instance
    """
    # Convert instances and prices to numpy array
    # filter prices that 
    y0=np.array(prices)  
    x=np.array(instances)[y0>0]
    y=y0[y0>0]
    # return -1 if no price availabe
    if len(y)==0:
        return -1
    # calling interpolate.interp1d using linear interpolation 
    # with extrapolatin 
    price=interpolate.interp1d(x, y, fill_value='extrapolate')(n)
    return float(price)

In [ ]:
def interpolate_prices1(n:int, instances:List[int], prices:List[float])->float:
    """
    interpolate using scipy.interpolate.
    :params:
      n : instance to find price
      instances: previous instances
      prices: previous prices
    :return:
      price for the instance
    """
    # x, y are lists to collect previous instances and prices
    x,y=[],[]
    # use idx to locate the index where the entry of instances
    # just passes n
    idx=0    
    for num, price in zip(instances, prices):
        if price>0:
            x.append(num)
            y.append(price)
            if num<n:
                idx+=1
            # early return when instance matches
            elif num==n:
                return price
            # bigger instance found 
            # idx >1 is required for extrapolation from beginning(left)
            elif idx>1:
                break 

    # return -1 if no price availalbe
    if len(y)==0:
        return -1
    # return the price if only one price available.    
    if len(y)==1:
        return y[0]
           
    # points for extrapolation from the right
    if idx==len(x):
        i0,i1=-2,-1
    #  points for extrapolation from the left
    elif idx==0:
        i0,i1=0,1
    # points for interpolation
    else:
        i0,i1=idx-1, idx

    # linear interpolation
    p=(y[i1]-y[i0])/(x[i1]-x[i0])*(n-x[i0])+y[i0]

    return p

    
        
         
    

In [ ]:
# test computation
for n in [2,5 ,7, 9, 15,17, 47 ]:
    p0=interpolate_prices0(n, instances, prices)
    p1=interpolate_prices1(n, instances, prices)
    print(f"Interpolated price using scipy:{p0:.2f}")
    print(f"Interpolated price using buld-in:{p1:.2f}")



Interpolated price using scipy:101.93
Interpolated price using list:101.93
Interpolated price using scipy:105.13
Interpolated price using list:105.13
Interpolated price using scipy:107.26
Interpolated price using list:107.26
Interpolated price using scipy:109.43
Interpolated price using list:109.43
Interpolated price using scipy:116.20
Interpolated price using list:116.20
Interpolated price using scipy:118.53
Interpolated price using list:118.53
Interpolated price using scipy:159.86
Interpolated price using list:159.86


In [ ]:
# test performance
%%timeit
for n in [2,5 ,7, 9, 15,17 ]:
    p0=interpolate_prices0(n, instances, prices)

1000 loops, best of 3: 595 µs per loop


In [ ]:
%%timeit
for n in [2,5 ,7, 9, 15,17 ]:
    p1=interpolate_prices1(n, instances, prices)

10000 loops, best of 3: 63.5 µs per loop


## Problem II Simple text query

Solution: 
* Use Counter memorize the word counts for sentences and phrases
* Check the phrases word counts are included in the those of the senctences. 
 
Complexity:
* The time complexity  is O(nN+2mnM), while N, M are the lengths of the sentence
 and the phrase, n and m for the number of sentences and queries. O(M),O(N) for the word count scans. O(M) for checking the coverage of a query in a sentence.
* The space complexity is O(nN+nM) for the word counters.

In [ ]:
from collections import Counter, defaultdict
from typing import List
def simple_query( sentences:str, phrases:str)->List[str]:
    # Count words in sentences and queries
    sentence_counters=[ Counter(s.split())  for s in sentences]
    phrase_counters=  [ Counter(p.split())  for p in phrases]
 
    res=[]
    for p in phrase_counters:
        # check if a sentence contains all the words of a query
        # by difference the counter of query to sentence
        item=[i for i, s in enumerate( sentence_counters) if not p-s ]
        res.append( item if item else [-1])

    return res                                



In [ ]:
#test
sentences=["jim likes mary","kate likes tom","tom does not like jim"]
queries=["jim tom","likes"]
simple_query(sentences, queries)

[[2], [0, 1]]

# Problem III Song pairs

Solution:
* Define a dictionary to save the count of remainder moded by 60
* Iterate the list of times, Check if the its counter part exisites in the dictionary
* increase the remainder

Complexity:
* The time complexity is O(N), whilce N is the lengh of the list,
 since it is one time scan.
* The space complexity if O(1), since the dictionary lengh is maximal 60.


In [ ]:
from collections import  defaultdict
def song_pairs(times: List[int]) -> int:
    """
    :params:
      times : list of song duration.

    :return: number of pairs

    """
    # dictionary for the remainder of 60
    hash = defaultdict(int)
    res = 0
    for t in times:
        r = t%60
        # count the number of counter part in dictionary
        if r==0:
            res+=hash[0]
        else:
            res+=hash[60-r]
        # increase the count the remainder r
        hash[r]+=1
    return res

In [ ]:
#test
times=[30,20,150,100,40]
song_pairs(times)

3